In [1]:
from dotenv import load_dotenv
import asyncio

load_dotenv()

True

In [ ]:
import os

In [11]:
import requests
import json


def stream_with_reasoning(question, api_token):
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_token}",
        "Content-Type": "application/json",
    }

    payload = {
        "model": "deepseek/deepseek-r1",
        "messages": [{"role": "user", "content": question}],
        "stream": True,
        "include_reasoning": True,
    }

    buffer = ""
    is_reasoning = True  # Track whether we're in reasoning or content phase
    print("Reasoning:", end="\n\n", flush=True)

    with requests.post(url, headers=headers, json=payload, stream=True) as r:
        for chunk in r.iter_content(chunk_size=1024, decode_unicode=True):
            buffer += chunk
            while True:
                try:
                    line_end = buffer.find("\n")
                    if line_end == -1:
                        break

                    line = buffer[:line_end].strip()
                    buffer = buffer[line_end + 1 :]

                    if line.startswith("data: "):
                        data = line[6:]
                        if data == "[DONE]":
                            break

                        try:
                            data_obj = json.loads(data)
                            delta = data_obj["choices"][0].get("delta", {})

                            # Check for phase transition
                            if (
                                is_reasoning
                                and "content" in delta
                                and delta.get("content")
                            ):
                                is_reasoning = False
                                print("\n\nResponse:", end="\n\n", flush=True)

                            # Handle reasoning phase
                            if is_reasoning and "reasoning" in delta:
                                reasoning = delta["reasoning"]
                                if reasoning and reasoning.strip():
                                    print(reasoning, end="", flush=True)

                            # Handle content phase
                            elif not is_reasoning and "content" in delta:
                                content = delta["content"]
                                if content and content.strip():
                                    print(content, end="", flush=True)

                        except json.JSONDecodeError:
                            pass
                except Exception as e:
                    print(f"\nError: {str(e)}")
                    break


question = "How would you build the tallest building ever?"
api_token = os.getenv("OPENROUTER_API_KEY")
stream_with_reasoning(question, api_token)

Reasoning:

Okay, so I need to figure out how to build the tallest building ever. Hmm, where do I even start?

KeyboardInterrupt: 

In [15]:
import requests
import json
from typing import Optional, List, Dict, Any, Iterator


class ChatCompletion:
    def __init__(self, response_data: Dict[str, Any]):
        self.choices = response_data.get("choices", [])
        self.id = response_data.get("id")
        self.model = response_data.get("model")
        self.created = response_data.get("created")
        self.usage = response_data.get("usage")
        self.response_type = response_data.get("response_type", "content")


class ChatCompletionChunk:
    def __init__(self, chunk_data: Dict[str, Any], response_type: str = "content"):
        self.choices = chunk_data.get("choices", [])
        self.id = chunk_data.get("id")
        self.model = chunk_data.get("model")
        self.created = chunk_data.get("created")
        self.response_type = response_type


class AsyncOpenAIOpenRouter:
    def __init__(self, api_key: str, base_url: str = "https://openrouter.ai/api/v1"):
        self.api_key = api_key
        self.base_url = base_url.rstrip("/")
        self.chat = self.Chat(self)

    class Chat:
        def __init__(self, client):
            self.client = client
            self.completions = self

        def create(
            self,
            model: str,
            messages: List[Dict[str, str]],
            stream: bool = False,
            include_reasoning: bool = False,
            **kwargs,
        ) -> Iterator[ChatCompletionChunk]:
            url = f"{self.client.base_url}/chat/completions"
            headers = {
                "Authorization": f"Bearer {self.client.api_key}",
                "Content-Type": "application/json",
            }

            payload = {
                "model": model,
                "messages": messages,
                "stream": stream,
                "include_reasoning": include_reasoning,
                **kwargs,
            }

            response = requests.post(url, headers=headers, json=payload, stream=stream)
            response.raise_for_status()

            if not stream:
                return ChatCompletion(response.json())

            buffer = ""
            is_reasoning = True if include_reasoning else False

            for chunk in response.iter_content(chunk_size=1024, decode_unicode=True):
                buffer += chunk
                while True:
                    try:
                        line_end = buffer.find("\n")
                        if line_end == -1:
                            break

                        line = buffer[:line_end].strip()
                        buffer = buffer[line_end + 1 :]

                        if line.startswith("data: "):
                            data = line[6:]
                            if data == "[DONE]":
                                break

                            try:
                                data_obj = json.loads(data)
                                delta = data_obj["choices"][0].get("delta", {})

                                # Handle reasoning phase
                                if is_reasoning and "reasoning" in delta:
                                    if (
                                        delta["reasoning"]
                                        and delta["reasoning"].strip()
                                    ):
                                        chunk_data = {
                                            "choices": [
                                                {
                                                    "delta": {
                                                        "reasoning": delta["reasoning"]
                                                    }
                                                }
                                            ]
                                        }
                                        yield ChatCompletionChunk(
                                            chunk_data, "reasoning"
                                        )

                                # Check for phase transition
                                elif is_reasoning and "content" in delta:
                                    is_reasoning = False

                                # Handle content phase
                                elif not is_reasoning and "content" in delta:
                                    if delta["content"] and delta["content"].strip():
                                        chunk_data = {
                                            "choices": [
                                                {"delta": {"content": delta["content"]}}
                                            ]
                                        }
                                        yield ChatCompletionChunk(chunk_data, "content")

                            except json.JSONDecodeError:
                                pass
                    except Exception as e:
                        print(f"\nError: {str(e)}")
                        break


# Example usage:
if __name__ == "__main__":
    import os

    client = AsyncOpenAIOpenRouter(
        api_key=os.getenv("OPENROUTER_API_KEY"),
    )

    question = "How would you build the tallest building ever?"

    response = client.chat.completions.create(
        model="deepseek/deepseek-r1",
        messages=[{"role": "user", "content": question}],
        stream=True,
        include_reasoning=True,
    )

    for chunk in response:
        if chunk.response_type == "reasoning":
            print(chunk.choices[0]["delta"]["reasoning"], end="", flush=True)
        else:
            print(chunk.choices[0]["delta"]["content"], end="", flush=True)

Okay, so I need to figure out how to build the tallest building ever. Let me start by thinking about what makes a building tall. First, the structure has to support its own weight and resist forces like wind and earthquakes. Maybe using strong materials like steel and concrete? But I remember that as buildings get taller, the materials need to be both strong and light. Maybe new materials like carbon fiber composites could help?Then there's the foundation. The taller the building, the more important the foundation is. It has to support all that weight. Maybe deep foundations with piles driven far into the ground? Or maybe a bedrock foundation if available. But what if the ground isn't stable? Maybe some kind of floating foundation? Not sure.

Wind is a big issue. Tall buildings sway, so they need systems to counteract that. I've heard of tuned mass dampers, like in the Taipei101. Maybe incorporating those. Also, the shape of the building could help reduce wind resistance. Tapered desig

KeyboardInterrupt: 